In [14]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools.render import render_text_description
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf

In [2]:
load_dotenv(find_dotenv("../.env"))

True

In [3]:
# llm=ChatGoogleGenerativeAI(model="gemini-2.0-flash-001")
llm=ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
prompt=ChatPromptTemplate.from_template(
    template="Tell me a 200 words story on {topic}"
)

In [5]:
chain=prompt|llm

<h2> Input Schema </h2>

<h5>The Input Schema of the chain is the input schema of its first part, the prompt</h5>

In [6]:
chain.input_schema.model_json_schema()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [7]:
prompt.input_schema.model_json_schema()

{'properties': {'topic': {'title': 'Topic', 'type': 'string'}},
 'required': ['topic'],
 'title': 'PromptInput',
 'type': 'object'}

In [8]:
# Not the same as the the chain and the prompt
llm.input_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'},
         {'additionalProperties': True, 'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'additionalProperties': True,
     'title': 'Additional Kwargs',
     'type': 'object'},
    'response_metadata': {'additionalProperties': True,
     'title': 'Response Metadata',
     'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default'

<h3>Output Schema</h3>

<h5>The Output Schema of the chain is the output schema of its last part, in this case a ChatModel, which outputs a ChatMessage</h5>

In [9]:
chain.output_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'},
         {'additionalProperties': True, 'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'additionalProperties': True,
     'title': 'Additional Kwargs',
     'type': 'object'},
    'response_metadata': {'additionalProperties': True,
     'title': 'Response Metadata',
     'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default'

In [10]:
llm.output_schema.model_json_schema()

{'$defs': {'AIMessage': {'additionalProperties': True,
   'description': 'Message from an AI.\n\nAIMessage is returned from a chat model as a response to a prompt.\n\nThis message represents the output of the model and consists of both\nthe raw output as returned by the model together standardized fields\n(e.g., tool calls, usage metadata) added by the LangChain framework.',
   'properties': {'content': {'anyOf': [{'type': 'string'},
      {'items': {'anyOf': [{'type': 'string'},
         {'additionalProperties': True, 'type': 'object'}]},
       'type': 'array'}],
     'title': 'Content'},
    'additional_kwargs': {'additionalProperties': True,
     'title': 'Additional Kwargs',
     'type': 'object'},
    'response_metadata': {'additionalProperties': True,
     'title': 'Response Metadata',
     'type': 'object'},
    'type': {'const': 'ai',
     'default': 'ai',
     'title': 'Type',
     'type': 'string'},
    'name': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
     'default'

<h3>Stream</h3>

In [11]:
for s in chain.stream(input={'topic':'bear'},):
    print(s.content,end="",flush=True)

There once lived a beautiful bear in the heart of the forest named Ben. Ben was well-known for his gentle nature and kind heart. He spent his days exploring the woods, playing with his friends, and enjoying the simple pleasures of nature.

One day, as Ben was taking a walk through the forest, he stumbled upon a group of hunters. They were setting traps and preparing to catch innocent animals for their own selfish purposes. Ben's heart sank as he realized the danger his friends were in.

Filled with determination, Ben decided to take action. He bravely confronted the hunters, standing between them and the defenseless animals. With a fierce roar, he warned them to leave the forest and never return.

The hunters, taken aback by Ben's courage and strength, quickly retreated, never to be seen again. Ben had saved his friends and protected the forest from harm.

From that day on, Ben was hailed as a hero among the animals of the forest. He continued to roam the woods, watching over his frien

<h3>Invoke</h3>

In [15]:
chain.invoke(input={"topic":"bear"}).content

"Once in a dense forest, there lived a large and fearsome bear named Brawn. Despite his intimidating appearance, Brawn was actually a kind-hearted creature who cared deeply for his fellow forest animals. He would often help them gather food, protect them from predators, and even play games with the younger ones.\n\nOne day, a group of hunters entered the forest, determined to capture Brawn and sell him to a circus. The other animals were terrified and didn't know what to do. But Brawn knew he had to protect his friends, so he devised a plan.\n\nHe led the hunters on a wild chase through the forest, using his speed and agility to outrun them at every turn. As the hunters grew tired and frustrated, Brawn snuck back to the other animals and told them to hide in a nearby cave.\n\nThe hunters eventually gave up and left the forest, defeated and empty-handed. The animals emerged from their hiding spot and cheered for their brave friend, Brawn the bear.\n\nFrom that day on, Brawn was hailed a

<h3>Batch</h3>

In [16]:
chain.batch(inputs=[
    {"topic":"bear"},
    {"topic":"Virat Kohli"}
])

[AIMessage(content='Long ago, in a dense forest, there lived a fearsome bear named Grizzly. Grizzly was known throughout the forest as the largest and fiercest bear, striking fear into the hearts of all the creatures that lived there.\n\nOne day, a group of hunters came to the forest in search of a great challenge. They had heard tales of Grizzly and were eager to test their skills against such a powerful beast. The hunters set traps and laid bait, hoping to catch the mighty bear.\n\nBut Grizzly was no fool. He sensed the danger and cunningly avoided the traps, outsmarting the hunters at every turn. The hunters grew frustrated, determined to capture the elusive bear.\n\nAs the days passed, Grizzly grew tired of the constant threat to his life. He decided it was time to teach the hunters a lesson they would never forget. With a mighty roar, Grizzly charged at the hunters, sending them running in fear.\n\nFrom that day on, the hunters knew to never underestimate the power and intelligenc

In [17]:
# Setting Concurrency

chain.batch(
    inputs=[
    {"topic":"bear"},
    {"topic":"Virat Kohli"},
    ],
    config={
        "max_concurrency":5
    }
)

[AIMessage(content='Once upon a time in a dense forest, there lived a lonely bear named Barnaby. He was known for his massive size and strength, but also for his gentle and kind nature. Despite his intimidating appearance, Barnaby was loved by all the animals in the forest.\n\nOne day, Barnaby stumbled upon a group of hunters who were looking for the perfect trophy to take home. The hunters were determined to capture Barnaby and sell him to a zoo for money. Fearful for his life, Barnaby knew he had to do something to protect himself.\n\nWith a heavy heart, Barnaby decided to leave the forest and seek refuge in the nearby mountains. As he roamed the rugged terrain, he faced many challenges and obstacles along the way. But through sheer determination and perseverance, Barnaby finally found a safe haven where he could live in peace and solitude.\n\nYears passed, and the forest animals never forgot their beloved Barnaby. They would often visit him in the mountains, bringing him gifts and s

<h3>Async Stream</h3>

In [21]:
async for s in chain.astream(input={'topic':'bear'}):
    print(s.content,end="",flush=True)

In the heart of the forest lived a mighty brown bear named Leo. He was the largest and strongest bear in the woods, and he ruled over all the other animals with a firm but fair paw.

Leo was a kind and gentle leader, always looking out for the well-being of his fellow creatures. He would often roam the forest, making sure everyone had enough food to eat and a warm den to sleep in. The other animals respected and admired him, knowing they were safe under his protection.

However, one day a group of hunters arrived in the forest, hoping to capture Leo and take him away to a zoo. The animals were terrified, but Leo stood his ground, ready to defend his home and his friends at any cost.

A fierce battle ensued, with Leo using all his strength and courage to protect his fellow creatures. In the end, the hunters were no match for the mighty bear, and they were forced to retreat from the forest.

The other animals cheered and celebrated their hero, grateful for his bravery and loyalty. And fr

<h3> Async Invoke </h3>

In [22]:
await chain.ainvoke(input={"topic":"Rohit Sharma"})

AIMessage(content='Rohit Sharma was a young man with a dream. From a small town in India, he had always been passionate about cricket. He spent hours practicing on dusty fields, honing his skills and perfecting his technique.\n\nDespite facing numerous challenges and setbacks along the way, Rohit never gave up on his dream. He worked hard, pushing himself to be better each day. His perseverance paid off when he was selected to play for the Indian national team.\n\nRohit quickly made a name for himself as a talented and explosive batsman. His ability to score big runs and lead his team to victory earned him the nickname "Hitman." He was loved by fans and respected by his teammates for his dedication and work ethic.\n\nWith each match, Rohit continued to improve and break records. He became the first cricketer in the world to score three double centuries in One Day Internationals. His talent and determination set him apart from his peers, making him one of the most formidable players in 

<h3>Async Batch </h3>

In [23]:
await chain.abatch(inputs=[{"topic":"Sachin Tendulkar"},{"topic":"Shane Warne"}])

[AIMessage(content="Sachin Tendulkar was undeniably one of the greatest cricketers of all time. His story began in Mumbai, where he showed immense talent and passion for the sport from a very young age. As a child, he would spend hours practicing in the streets of his neighborhood, dreaming of one day playing for the Indian national team.\n\nHis hard work and perseverance paid off when he made his international debut at the age of just 16. Despite facing much older and more experienced opponents, Sachin quickly proved himself to be a prodigy, scoring centuries and breaking records with astonishing regularity.\n\nOver the course of his career, Sachin became a national hero in India, with millions of fans around the world tuning in to watch him play. His skill with the bat was unmatched, and he faced some of the best bowlers in the world with grace and determination.\n\nBut it wasn't just his skill on the field that made Sachin so beloved. He was known for his humility, his sportsmanship

<h3>Event Reference</h3>

In [27]:
template="""
Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt=ChatPromptTemplate.from_template(template=template)

vectorStore=FAISS.from_texts(
    texts=["Ritish has a 18 month old son !",
           "Rohit is Ritish's Younger Brother",
           "Eklavya is Ritish's Son"
           "Rekha is the name of Ritish's Mother",
           "Anirban is Ritish's Father"],
    embedding=OpenAIEmbeddings())

retriever=vectorStore.as_retriever()

outputParser=StrOutputParser()

In [29]:
retrievalChain=RunnablePassthrough.assign(context=itemgetter("question")|retriever.with_config(run_name="Docs") ) | prompt | llm.with_config(run_name="my_llm") | outputParser

In [30]:
question={"question":"Who is Anirban's Wife and who is Eklavya's Uncle?"}

In [31]:
print(retrievalChain.invoke(input=question))

Anirban's wife is Rekha, and Eklavya's uncle is Rohit.


In [32]:
async for event in retrievalChain.astream_events(input=question,
                                                 version="v1",
                                                 include_names=["Docs","my_llm"]
                                                ):
    print(event,"\n")

{'event': 'on_retriever_start', 'name': 'Docs', 'run_id': '9c264c6c-416a-40e9-a6e0-e1cd7a1da3ce', 'tags': ['seq:step:2', 'FAISS', 'OpenAIEmbeddings'], 'metadata': {'ls_retriever_name': 'vectorstore', 'ls_vector_store_provider': 'FAISS', 'ls_embedding_provider': 'OpenAIEmbeddings'}, 'data': {'input': {'query': "Who is Anirban's Wife and who is Eklavya's Uncle?"}}, 'parent_ids': []} 

{'event': 'on_retriever_end', 'name': 'Docs', 'run_id': '9c264c6c-416a-40e9-a6e0-e1cd7a1da3ce', 'tags': ['seq:step:2', 'FAISS', 'OpenAIEmbeddings'], 'metadata': {'ls_retriever_name': 'vectorstore', 'ls_vector_store_provider': 'FAISS', 'ls_embedding_provider': 'OpenAIEmbeddings', 'LANGSMITH_ENDPOINT': 'https://api.smith.langchain.com', 'LANGSMITH_PROJECT': 'tutorial-project', 'LANGSMITH_TRACING': 'true', 'revision_id': 'da0f6c7-dirty'}, 'data': {'input': {'query': "Who is Anirban's Wife and who is Eklavya's Uncle?"}, 'output': {'documents': [Document(id='070cd071-88b3-4a2c-b2bb-3a47fe0766a3', metadata={}, pa

In [33]:
async for event in retrievalChain.astream_events(input=question,
                                                 version="v1",
                                                 include_names=["Docs","my_llm"]
                                                ):
    kind=event['event']
    if kind=="on_chat_model_stream":
        print(event["data"]["chunk"].content,end="|")
    elif kind=="on_chat_model_start":
        print("\nStreaming LLM")
    elif kind=="on_chat_model_end":
        print("Done Streaming LLM.")
    elif kind=="on_retriever_end":
        print("Retrieved the following Documents:")
        print(event["data"]["output"]["documents"])
    elif kind=="on_tool_end":
        print(f"Ended Tool: {event['name']}")

Retrieved the following Documents:
[Document(id='070cd071-88b3-4a2c-b2bb-3a47fe0766a3', metadata={}, page_content="Anirban is Ritish's Father"), Document(id='7232316e-dfe2-41d6-9396-bb59602ea9f0', metadata={}, page_content="Eklavya is Ritish's SonRekha is the name of Ritish's Mother"), Document(id='9ed7ab3d-a152-43cd-91e4-aed21eda3cd1', metadata={}, page_content="Rohit is Ritish's Younger Brother"), Document(id='b3c9243c-1c52-467c-a322-322046b0b5b3', metadata={}, page_content='Ritish has a 18 month old son !')]

Streaming LLM
|An|ir|ban|'s| wife| is| Re|k|ha| and| E|kl|avy|a|'s| uncle| is| Roh|it|.||Done Streaming LLM.


In [34]:
async for event in retrievalChain.astream_events(input=question,
                                                 version="v2",
                                                 include_names=["Docs","my_llm"]
                                                ):
    kind=event['event']
    if kind=="on_chat_model_stream":
        print(event["data"]["chunk"].content,end="|")
    elif kind=="on_chat_model_start":
        print("\nStreaming LLM")
    elif kind=="on_chat_model_end":
        print("Done Streaming LLM.")
    elif kind=="on_retriever_end":
        print("Retrieved the following Documents:")
        print(event['data']['output'])
        # print(event["data"]["output"]["documents"])
    elif kind=="on_tool_end":
        print(f"Ended Tool: {event['name']}")

Retrieved the following Documents:
[Document(id='070cd071-88b3-4a2c-b2bb-3a47fe0766a3', metadata={}, page_content="Anirban is Ritish's Father"), Document(id='7232316e-dfe2-41d6-9396-bb59602ea9f0', metadata={}, page_content="Eklavya is Ritish's SonRekha is the name of Ritish's Mother"), Document(id='9ed7ab3d-a152-43cd-91e4-aed21eda3cd1', metadata={}, page_content="Rohit is Ritish's Younger Brother"), Document(id='b3c9243c-1c52-467c-a322-322046b0b5b3', metadata={}, page_content='Ritish has a 18 month old son !')]

Streaming LLM
|An|ir|ban|'s| wife| is| Re|k|ha| and| E|kl|avy|a|'s| uncle| is| Roh|it|.||Done Streaming LLM.


In [44]:
async for chunk in retrievalChain.astream_log(input=question,include_names=["Docs"]):
    print("-"*40)
    print(chunk)

----------------------------------------
RunLogPatch({'op': 'replace',
  'path': '',
  'value': {'final_output': None,
            'id': '51cd7d04-383e-45db-b4a0-29d319fa3f20',
            'logs': {},
            'name': 'RunnableSequence',
            'streamed_output': [],
            'type': 'chain'}})
----------------------------------------
RunLogPatch({'op': 'add',
  'path': '/logs/Docs',
  'value': {'end_time': None,
            'final_output': None,
            'id': '1b6c6dff-cadb-4a15-9366-2e8ebddc44d4',
            'metadata': {'ls_embedding_provider': 'HuggingFaceEmbeddings',
                         'ls_retriever_name': 'vectorstore',
                         'ls_vector_store_provider': 'FAISS'},
            'name': 'Docs',
            'start_time': '2025-06-04T13:31:28.709+00:00',
            'streamed_output': [],
            'streamed_output_str': [],
            'tags': ['seq:step:2', 'FAISS', 'HuggingFaceEmbeddings'],
            'type': 'retriever'}})
---------------

In [71]:
## Stream Incremental RunState with diff=False
async for chunk in retrievalChain.astream_log(input=question,include_names=["Docs"],diff=False):
    print("-"*40)
    print(chunk)

----------------------------------------
RunLog({'final_output': None,
 'id': '285c7584-ffc6-4fda-8de4-3a2a96530f6c',
 'logs': {},
 'name': 'RunnableSequence',
 'streamed_output': [],
 'type': 'chain'})
----------------------------------------
RunLog({'final_output': None,
 'id': '285c7584-ffc6-4fda-8de4-3a2a96530f6c',
 'logs': {'Docs': {'end_time': None,
                   'final_output': None,
                   'id': 'd6cba482-cd44-4f72-bab0-7fe738b5d482',
                   'metadata': {'ls_embedding_provider': 'HuggingFaceEmbeddings',
                                'ls_retriever_name': 'vectorstore',
                                'ls_vector_store_provider': 'FAISS'},
                   'name': 'Docs',
                   'start_time': '2024-09-24T20:44:13.157+00:00',
                   'streamed_output': [],
                   'streamed_output_str': [],
                   'tags': ['seq:step:2', 'FAISS', 'HuggingFaceEmbeddings'],
                   'type': 'retriever'}},
 'name':

<h3>Runnable Parallel</h3>

In [35]:
chain1=ChatPromptTemplate.from_template(template="Tell Me a Joke about {topic}")|llm
chain2=ChatPromptTemplate.from_template(template="Write a Poem on the Following {subject}")|llm

In [36]:
combinedChain=RunnableParallel(joke=chain1,poem=chain2)

In [37]:
start=time.time()
print(chain1.invoke(input={"topic":"Titanic"}).content)
print(f"Took {time.time()-start} Seconds")

Why did the iceberg go to therapy?

Because it had some serious emotional issues to work through after sinking the Titanic.
Took 0.9242572784423828 Seconds


In [38]:
start=time.time()
print(chain2.invoke(input={"subject":"Titanic"}).content)
print(f"Took {time.time()-start} Seconds")

Titanic, oh mighty ship of steel,
A wonder of the age, so grand and real.
Set sail on a fateful night,
Destined for glory, bathed in light.

But fate had other plans in store,
As the iceberg loomed, a deadly roar.
The collision was swift, the damage done,
And the unsinkable ship, began to run.

Panic erupted, chaos reigned,
As the passengers faced a fate unexplained.
The lifeboats filled, the cries were heard,
As the Titanic slowly, tragically, was interred.

The music played, the lights went dim,
As the ocean claimed its solemn hymn.
The stars bore witness to the tragic end,
Of a ship so grand, that fate did bend.

Yet in the midst of tragedy and loss,
Stories of heroism, of love, across
The waves of time, they still endure,
A testament to the human spirit, pure.

Titanic, a name that will forever be,
A reminder of the fragility of the sea.
A lesson learned, not to be lost,
In the depths of the ocean, at any cost.
Took 1.8193910121917725 Seconds


In [52]:
start=time.time()
print(combinedChain.invoke(input={"subject":"Titanic","topic":"Titanic"}))
print(f"Took {time.time()-start} Seconds")

{'joke': AIMessage(content='Why did the iceberg get a bad reputation?\n\nBecause it sank a lot of relationships.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-001', 'safety_ratings': []}, id='run-8c045317-3103-4ee4-8f4d-48fe5368d52a-0', usage_metadata={'input_tokens': 6, 'output_tokens': 19, 'total_tokens': 25, 'input_token_details': {'cache_read': 0}}), 'poem': AIMessage(content="The Dream of Iron, forged in fire's heart,\nA floating palace, set to play its part.\nTitanic, queen of ocean, bold and grand,\nA marvel built by human skill and hand.\n\nShe slipped her moorings, Belfast left behind,\nA promise whispered on the salty wind.\nOf gilded halls and dinners, rich and deep,\nWhere fortunes mingled, secrets they would keep.\n\nThe steerage huddled, hopes within their breast,\nA new life beckoned, westward to the West.\nThey dreamt of freedom, land and open sky,\nUnburd

In [40]:
start=time.time()
RunnableParallel({'joke':chain1,'poem':chain2}).invoke(input={"subject":"Titanic","topic":"Titanic"})
print(f"Took {time.time()-start} Seconds")


Took 2.749480724334717 Seconds


In [42]:
combinedChain

{
  joke: ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell Me a Joke about {topic}'), additional_kwargs={})])
        | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000029F6B5C8410>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000029F4FC91050>, root_client=<openai.OpenAI object at 0x0000029F6B1FF3D0>, root_async_client=<openai.AsyncOpenAI object at 0x0000029F6B5C8750>, model_kwargs={}, openai_api_key=SecretStr('**********')),
  poem: ChatPromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, template='Write a Poem on the Following {subject}'), additional_kwargs={})])
        

<h3> Parallelism on Batches</h3>

In [43]:
start=time.time()
print(chain1.batch(inputs=[{"topic":"Titanic"},{"topic":"Kolkata"}]))
print(f"Took {time.time()-start} Seconds")

[AIMessage(content="Why did the iceberg break up with the Titanic? \n\nIt just couldn't handle the pressure.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 14, 'total_tokens': 33, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CxeWyQeCD0mue3w2hQS4uReGKEBEK', 'finish_reason': 'stop', 'logprobs': None}, id='run--d0cb9e73-1598-4c13-b4f1-0dbe60e05efa-0', usage_metadata={'input_tokens': 14, 'output_tokens': 19, 'total_tokens': 33, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), AIMessage(content='Why did the chicken go to Kolkata?\n\nTo get to the other side of the Howrah Bridge!', additional_kwargs={'refusal': None}, response_

In [44]:
start=time.time()
print(chain2.batch(inputs=[{"subject":"Titanic"},{"subject":"Kolkata"}]))
print(f"Took {time.time()-start} Seconds")

[AIMessage(content="In the dead of night, a towering ship set sail\nThe Titanic, grand and magnificent, its beauty did prevail\nA marvel of engineering, a testament to man's might\nBut little did they know, it would face a fateful fight\n\nAcross the cold Atlantic, she glided with grace\nA symbol of luxury, a vision of elegance and pace\nBut lurking beneath the surface, danger lay in wait\nAn iceberg stood silent, sealing the ship's dark fate\n\nIn an instant, the world was turned upside down\nAs the Titanic collided with a force unseen and profound\nPanic and chaos ensued, as the ship began to sink\nThe screams of the terrified, the water turning pink\n\nBrave souls rose up, to help those in need\nBut the icy waters took their toll, a price no one could heed\nThe Titanic, once mighty, now lay broken and still\nA tragic reminder, of the power of nature's will\n\nWe remember the lives lost, the courage and the pain\nThe Titanic, a legend, will forever remain\nA lesson to us all, of hubr

In [45]:
start=time.time()
print(combinedChain.batch(inputs=[
                                    {"subject":"Titanic","topic":"Titanic"},  # Joke on Titanic and Poem on Titanic
                                    {"subject":"Kolkata","topic":"Kolkata"}  # Joke on Kolkata and Poem on Kolkata
                          ]))
print(f"Took {time.time()-start} Seconds")

[{'joke': AIMessage(content="Why did the iceberg refuse to apologize to the Titanic?\n\nBecause it didn't want to break the ice!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 14, 'total_tokens': 35, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CxeXUqqFACkBnpvKMphFZoBSE3IuY', 'finish_reason': 'stop', 'logprobs': None}, id='run--903fef6a-a6b2-4bb6-a426-03af330d992e-0', usage_metadata={'input_tokens': 14, 'output_tokens': 21, 'total_tokens': 35, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}), 'poem': AIMessage(content="Titanic, a ship of grandeur and grace\nSailing through the vast ocean space\nIts towering structure, a si

In [46]:
combinedChain.get_graph().print_ascii()

               +--------------------------+                
               | Parallel<joke,poem>Input |                
               +--------------------------+                
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+*                     +------------+     
                   ***               ***                   
                      ***         ***   

In [47]:
combinedChain.get_prompts()

[ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='Tell Me a Joke about {topic}'), additional_kwargs={})]),
 ChatPromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['subject'], input_types={}, partial_variables={}, template='Write a Poem on the Following {subject}'), additional_kwargs={})])]